In [34]:
import sys
import datetime
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from siftools.metrics import *
from siftools.sif_utils import get_data, get_sp_close



start_date = datetime.datetime(2015, 10, 1)
end_date = datetime.datetime(2020, 10, 23)

tickers = ['GOOGL']
data = get_data("equity_raw", tickers, start_date, end_date).set_index('date').sort_index()

# Normalize the data
percent_growth = pd.DataFrame([i / data['close'][0] for i in data['close']], index = data.index)
plt.plot(percent_growth)



In [46]:
''' This function takes in the Pandas data for a stock
and a smoothing value, and makes an EMA line
'''
def make_ema(percent_growth, smoothing):

    # Make EMA lines
    EMA = [1]

    for day in percent_growth[0][1:]:
        # https://www.investopedia.com/terms/e/ema.asp
        temp = day * (smoothing / (1 + len(percent_growth)))
        temp += (EMA[-1] * (1 - (smoothing / (1 + len(percent_growth)))))
        EMA.append(temp)

    EMA = pd.DataFrame(EMA, index = percent_growth.index).sort_index()

    plt.plot(percent_growth)
    plt.plot(EMA)

    data = percent_growth.to_numpy().flatten().tolist()
    EMA = EMA.to_numpy().flatten().tolist()
    
    return EMA

In [57]:
''' This function takes raw data and the EMA data from that raw data (both in list format)
and calculates the returns for them using a simple EMA momentum strategy'''
def calculate_returns(data, EMA):

    count = 1
    money = 1
    invested = 0
    returns = []

    for day in data[1:]:
        # Update invested money
        invested *= 1 + (day - data[count - 1])

        # Buy and Sell
        if invested == 0:
            if day > EMA[count]:
                # Simulate buy
                invested = money
                money = 0

        else:
            if day < EMA[count]:
                # Simulate sell
                money = invested
                invested = 0


        returns.append(money + invested)
        count += 1

    return returns


''' This portion creates profit graphs for varius smoothings factors,
from 0 - 100 inclusive in increments of 10. It also stores the ending
returns from those profit graphs so that they can be graphed later
with the smoothing factors.'''
totals = []
graphs = []
for i in range(20):
    ema = make_ema(percent_growth, 5 * (i+1))
    returns = calculate_returns(percent_growth.to_numpy().flatten().tolist(), ema)
    graphs.append(returns)
    totals.append(returns[-1])
    

In [59]:
# Graph scatter plot of smoothing factors vs totals
plt.scatter([5 * i for i in range(1, 21)], totals)